In [1]:
import os

In [2]:
%pwd

'e:\\ML_Projects\\Wine_Quality_Project\\research'

In [3]:
os.chdir("../")

In [4]:
# this code belong from entity/config_entity.py file

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [5]:
# this code belong from config/configuration.py file
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config
    

In [6]:
# this code belong from component/data_ingestion.py file
import os
import zipfile
from urllib import request
from pathlib import Path
from mlProject import logger  # adjust this import as per your project structure
from mlProject.utils.common import get_size  # make sure this is where your `get_size` is

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
    
        if not zipfile.is_zipfile(self.config.local_data_file):
            raise zipfile.BadZipFile("The downloaded file is not a valid zip file.")

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        logger.info(f"Extracted zip file to: {unzip_path}")


In [7]:
# this code belong from pipeline/stage_01_data_ingestion.py file
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-05-26 02:36:02,885: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-05-26 02:36:02,893: INFO: common: YAML file: params.yaml loaded successfully]
[2025-05-26 02:36:02,908: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-05-26 02:36:02,968: INFO: common: Created directory at: artifacts]
[2025-05-26 02:36:02,981: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-05-26 02:36:06,686: INFO: 2193420032: artifacts/data_ingestion/wine_data.zip downloaded! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EEB5:278F4:1F0C97A:221B7BD:68336316
Accept-Ranges: bytes
Date: